# Trabalho de Algoritmos Genéticos

## Problema Do Empacotamento (BPP do inglês *"Bin Packing Problem"*)

Dada uma quantidade inteira e positiva de pacotes/depósitos de capacidade $C$ e um set de $M$ itens  $I = [I_1, \cdots, I_M]$ de tamanhos $S = [S_1,\cdots,S_M]$, o problema consiste em empacotar todos os itens nos pacotes, de modo a não exceder a capacidade $C$, **minimizando** a quantidade $N$ de pacotes utilizados.

## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import pygad
import tensorflow.keras

import numpy as np
import pandas as pd
from itertools import combinations

## Modelando o Problema

- Gene: É um pacote.
    - É representado por um set de inteiros referentes à cada cada item;
- Indivíduo: É um conjunto de pacotes, que somam um total de $M$ elementos, cujos respectivos pesos não infringem a capacidade $C$.

In [2]:
def f(individuo):
    return len(individuo)

In [31]:
def FF(individuo, item, C, S):

    if individuo == []:
        individuo.append([item])
        return

    added = False
    for pacote in individuo:
        if sum([x for i, x in enumerate(S) if i in pacote]) + S[item] <= C:
            pacote.append(item)
            pacote.sort()
            added = True
            break

    if not added:
        individuo.append([item])
            

def FFD(individuo, itens, C, S):
    ordenados = [x[0] for x in sorted(
        [(itens[i], S[i]) for i in range(0, len(S))], 
        key = lambda x: x[1],
        reverse=True)]
    
    for item in ordenados:
        FF(individuo, item, C, S)


def MBS(individuo, item, C, S):

    if individuo == []:
        individuo.append([item])
        return

    added = False
    melhor_diferenca = C
    indice_melhor = 0
    for I, pacote in enumerate(individuo):
        diferenca = C - (sum([x for i, x in enumerate(S) if i in pacote]) + S[item])
        if diferenca < melhor_diferenca and diferenca >= 0:
            melhor_diferenca = diferenca
            indice_melhor = I
            added = True

    if added:
        individuo[indice_melhor].append(item)
        individuo[indice_melhor].sort()
    else:
        individuo.append([item])

def FleszarMBS(individuo, itens, C, S):
    ordenados = [x[0] for x in sorted(
        [(itens[i], S[i]) for i in range(0, len(S))], 
        key = lambda x: x[1],
        reverse=True)]
    for it in ordenados:
        MBS(individuo, it, C, S)

In [ ]:
def add(pacote, item, C, S):
    if (sum([x for i, x in enumerate(S) if i in pacote]) + S[item]) <= C:
        pacote.append(item)

In [35]:
def substituicao(individuo, Ta, Tb):
    Tc = [list(x) for x in combinations(Ta, 2)]

    S_vec = []
    for pacote in individuo:
        Sv = []
        Sv += [list(x) for x in combinations(pacote, 1)]
        Sv += [list(x) for x in combinations(pacote, 2)]
        Sv += [list(x) for x in combinations(pacote, 3)]

        S_vec.append(Sv)

    f_ind = f(individuo)
    lista_candidatos = []
    for candidato in S_vec:
        nTa = Ta.copy()
        nTa = [x for x in nTa if x not in candidato]
        for pct in individuo:
            nInd = individuo.copy()
            nInd.remove(pct)

            nInd.append(candidato)
            for item in pct:
                nTa.append(item)

            f_new = f(nInd)
            if f_new < f_ind:
                lista_candidatos.append(tuple(f_new, nTa, Tb.copy(), nInd))


    for candidato in Tb:
        nTb = Tb.copy()
        nTb.remove(candidato)
        nTa = Ta.copy()
        for pct in individuo:
            nInd = individuo.copy()
            nInd.remove(pct)

            nInd.append(candidato)
            for item in pct:
                nTa.append(item)

            f_new = f(nInd)
            if f_new < f_ind:
                lista_candidatos.append(tuple(f_new, nTa, nTb, nInd))
    
    melhor_candidato = min(lista_candidatos, key = lambda x: x[0])
    
        # TODO: PROCEDIMENTO EM DUAS ETAPAS COM O MELHOR CANDIDATO:
        #       Para Tb: tentar inserir o set INTEIRO usando FF (não criar novo set)
        #                Se nao der certo: inserir cada elemento com MBS
        #       Para Ta: FF
        
                
        
        

def cruzamento(pai1, pai2, C):
    idx1 = [1,3] # rand 2 de cima
    idx2 = [3]   # rand 1 de baixo mas sem elementos de pai1[indices1]

    offset = []
    offset += [pacote for indice, pacote in enumerate(pai1) if indice in idx1]
    offset += [pacote for indice, pacote in enumerate(pai2) if indice in idx2]
    
    Ta = []
    Tb = []

    # pacotes remanescentes em pai2
    for pacote in [pct for indice, pct in enumerate(pai2) if indice not in idx2]:
        novo_pacote = []
        for item in pacote:
            if not any(item in o_pct for o_pct in offset):
                novo_pacote.append(item)

        if len(novo_pacote) > 1:
            Tb.append(novo_pacote)
        elif len(novo_pacote) == 1:
            Ta.append(novo_pacote[0])
    




    

    return offset, crossover(pai2, pai1, C)

In [32]:
p1 = [[1,3,10],[2,9,11],[5,7,13,15],[4,6,14],[8,12]]
p2 = [[3,4,12],[6,7,11],[9,10],[1,5,8],[2,13,14]]